In [ ]:
import os
import sys
import glob
import subprocess
from collections import OrderedDict

import pandas as pd
import numpy as np
from tqdm import tqdm

sys.path.append('/home/alexandrearaujo/caffe/python/')
import caffe
import cv2

import tensorflow as tf



In [ ]:
# usage: test.py [-h] --gpu GPU --S S --L L --proto PROTO --weights WEIGHTS
#                --dataset DATASET --dataset_name DATASET_NAME --eval_binary
#                EVAL_BINARY --temp_dir TEMP_DIR [--multires] [--aqe AQE]
#                [--dbe DBE]

# G: gpu id
# S: size to resize the largest side of the images to. The model is trained with S=800, but different values may work better depending on the task.
# L: number of levels of the rigid grid. Model was trained with L=2, but different levels (e.g. L=1 or L=3) may work better on other tasks.
# PROTO: path to the prototxt. There are two prototxts included.
#   deploy_resnet101.prototxt relies on caffe being compiled with the normalization layer.
#   deploy_resnet101_normpython.prototxt does not have that requirement as it relies on the python implementation, but it may be slower as it is done on the cpu and does not implement backpropagation.
# WEIGHTS: path to the caffemodel
# DATASET: path to the dataset, for Oxford and Paris it is the directory that contains the jpg and lab folders.
# DATASET_NAME: either Oxford or Paris
# EVAL_BINARY: path to the compute_ap binary provided with Oxford and Paris used to compute the ap scores
# TEMP_DIR: a temporary directory to store features and scores

In [ ]:
queries = glob.glob('/media/hdd1/kaggle/landmark-retrieval-challenge/test_rescale/*')
index = glob.glob('/media/hdd1/kaggle/landmark-retrieval-challenge/index_rescale/*')

In [ ]:
class ImageHelper:
    def __init__(self, S, L, means):
        self.S = S
        self.L = L
        self.means = means

    def prepare_image_and_grid_regions_for_network(self, fname, roi=None):
        # Extract image, resize at desired size, and extract roi region if
        # available. Then compute the rmac grid in the net format: ID X Y W H
        I, im_resized = self.load_and_prepare_image(fname, roi)
        if self.L == 0:
            # Encode query in mac format instead of rmac, so only one region
            # Regions are in ID X Y W H format
            R = np.zeros((1, 5), dtype=np.float32)
            R[0, 3] = im_resized.shape[1] - 1
            R[0, 4] = im_resized.shape[0] - 1
        else:
            # Get the region coordinates and feed them to the network.
            all_regions = []
            all_regions.append(self.get_rmac_region_coordinates(im_resized.shape[0], im_resized.shape[1], self.L))
            R = self.pack_regions_for_network(all_regions)
        return I, R

    def get_rmac_features(self, I, R, net):
        net.blobs['data'].reshape(I.shape[0], 3, int(I.shape[2]), int(I.shape[3]))
        net.blobs['data'].data[:] = I
        net.blobs['rois'].reshape(R.shape[0], R.shape[1])
        net.blobs['rois'].data[:] = R.astype(np.float32)
        net.forward(end='rmac/normalized')
        return np.squeeze(net.blobs['rmac/normalized'].data)

    def load_and_prepare_image(self, fname, roi=None):
        # Read image, get aspect ratio, and resize such as the largest side equals S
        im = cv2.imread(fname)
        im_size_hw = np.array(im.shape[0:2])
        ratio = float(self.S)/np.max(im_size_hw)
        new_size = tuple(np.round(im_size_hw * ratio).astype(np.int32))
        im_resized = cv2.resize(im, (new_size[1], new_size[0]))
        # If there is a roi, adapt the roi to the new size and crop. Do not rescale
        # the image once again
        if roi is not None:
            roi = np.round(roi * ratio).astype(np.int32)
            im_resized = im_resized[roi[1]:roi[3], roi[0]:roi[2], :]
        # Transpose for network and subtract mean
        I = im_resized.transpose(2, 0, 1) - self.means
        return I, im_resized

    def pack_regions_for_network(self, all_regions):
        n_regs = np.sum([len(e) for e in all_regions])
        R = np.zeros((n_regs, 5), dtype=np.float32)
        cnt = 0
        # There should be a check of overflow...
        for i, r in enumerate(all_regions):
            try:
                R[cnt:cnt + r.shape[0], 0] = i
                R[cnt:cnt + r.shape[0], 1:] = r
                cnt += r.shape[0]
            except:
                continue
        assert cnt == n_regs
        R = R[:n_regs]
        # regs where in xywh format. R is in xyxy format, where the last coordinate is included. Therefore...
        R[:n_regs, 3] = R[:n_regs, 1] + R[:n_regs, 3] - 1
        R[:n_regs, 4] = R[:n_regs, 2] + R[:n_regs, 4] - 1
        return R

    def get_rmac_region_coordinates(self, H, W, L):
        # Almost verbatim from Tolias et al Matlab implementation.
        # Could be heavily pythonized, but really not worth it...
        # Desired overlap of neighboring regions
        ovr = 0.4
        # Possible regions for the long dimension
        steps = np.array((2, 3, 4, 5, 6, 7), dtype=np.float32)
        w = np.minimum(H, W)

        b = (np.maximum(H, W) - w) / (steps - 1)
        # steps(idx) regions for long dimension. The +1 comes from Matlab
        # 1-indexing...
        idx = np.argmin(np.abs(((w**2 - w * b) / w**2) - ovr)) + 1

        # Region overplus per dimension
        Wd = 0
        Hd = 0
        if H < W:
            Wd = idx
        elif H > W:
            Hd = idx

        regions_xywh = []
        for l in range(1, L+1):
            wl = np.floor(2 * w / (l + 1))
            wl2 = np.floor(wl / 2 - 1)
            # Center coordinates
            if l + Wd - 1 > 0:
                b = (W - wl) / (l + Wd - 1)
            else:
                b = 0
            cenW = np.floor(wl2 + b * np.arange(l - 1 + Wd + 1)) - wl2
            # Center coordinates
            if l + Hd - 1 > 0:
                b = (H - wl) / (l + Hd - 1)
            else:
                b = 0
            cenH = np.floor(wl2 + b * np.arange(l - 1 + Hd + 1)) - wl2

            for i_ in cenH:
                for j_ in cenW:
                    regions_xywh.append([j_, i_, wl, wl])

        # Round the regions. Careful with the borders!
        for i in range(len(regions_xywh)):
            for j in range(4):
                regions_xywh[i][j] = int(round(regions_xywh[i][j]))
            if regions_xywh[i][0] + regions_xywh[i][2] > W:
                regions_xywh[i][0] -= ((regions_xywh[i][0] + regions_xywh[i][2]) - W)
            if regions_xywh[i][1] + regions_xywh[i][3] > H:
                regions_xywh[i][1] -= ((regions_xywh[i][1] + regions_xywh[i][3]) - H)
        return np.array(regions_xywh).astype(np.float32)
    
class NormalizeLayer(caffe.Layer):
    def setup(self, bottom, top):
        assert len(bottom) == 1, 'This layer can only have one bottom'
        assert len(top) == 1, 'This layer can only have one top'
        self.eps = 1e-8

    def reshape(self, bottom, top):
        top[0].reshape(*bottom[0].data.shape)

    def forward(self, bottom, top):
        top[0].data[:] = bottom[0].data / np.expand_dims(self.eps + np.sqrt((bottom[0].data ** 2).sum(axis=1)), axis=1)

    def backward(self, top, propagate_down, bottom):
        raise NotImplementedError("Backward pass not supported with this implementation")


class AggregateLayer(caffe.Layer):
    def setup(self, bottom, top):
        assert len(bottom) == 1, 'This layer can only have one bottom'
        assert len(top) == 1, 'This layer can only have one top'

    def reshape(self, bottom, top):
        tmp_shape = list(bottom[0].data.shape)
        tmp_shape[0] = 1
        top[0].reshape(*tmp_shape)

    def forward(self, bottom, top):
        top[0].data[:] = bottom[0].data.sum(axis=0)

    def backward(self, top, propagate_down, bottom):
        raise NotImplementedError("Backward pass not supported with this implementation")


In [ ]:
S = 512
L = 2
means = np.array([103.93900299,  116.77899933,  123.68000031], dtype=np.float32)[None, :, None, None]

In [ ]:
image_helper = ImageHelper(S, L, means)

In [ ]:
proto  = '../submit/16_dir_desc/deploy_resnet101_normpython.prototxt'
weights  = '../submit/16_dir_desc/model.caffemodel'

In [ ]:
# Configure caffe and load the network
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Net(proto, weights, caffe.TEST)

In [ ]:
I, R = image_helper.prepare_image_and_grid_regions_for_network(index[2], roi=None)
R = R.astype('float32')

In [ ]:
print(I.shape)
print(R.shape)

In [ ]:
net.blobs['data'].reshape(I.shape[0], 3, int(I.shape[2]), int(I.shape[3]))
net.blobs['data'].data[:] = I
net.blobs['rois'].reshape(R.shape[0], R.shape[1])
net.blobs['rois'].data[:] = R.astype(np.float32)
net.forward(end='rmac/normalized')
res = net.blobs['rmac/normalized'].data
print(res.shape)

In [ ]:
print(net.blobs['data'].data.shape)

In [ ]:
batch_size = 5
net.blobs['data'].reshape(batch_size, 3, 512, 384)
net.blobs['rois'].reshape(batch_size, R.shape[0], R.shape[1])

shape = (1, 3, 512, 384)

for i in range(batch_size):

    I, R = image_helper.prepare_image_and_grid_regions_for_network(index[i], roi=None)
    R = R.astype('float32')
    if I.shape != shape:
        I = I.transpose(0, 1, 3, 2)
    print(I.shape)
    
    net.blobs['data'].data[i] = I
    net.blobs['rois'].data[i] = R

net.forward(end='pooled_rois/pca/normalized')
res_new = net.blobs['pooled_rois/pca/normalized'].data
print(res_new.shape)

In [ ]:
features_queries = np.vstack([
    np.vstack((features_queries[i], features_dataset[idx[i, :args.aqe]])).mean(axis=0) for i in range(len(features_queries))])

In [ ]:
for i in range(len(features_queries)):
    

In [1]:
a = [1, 2, 3]

In [2]:
a[:1]

[1]

In [ ]:
vec = image_helper.get_rmac_features(I, R, net)

In [ ]:
vec.shape

In [ ]:
import tensorflow as tf

In [ ]:
def parse_fn(example):
  "Parse TFExample records and perform simple data augmentation."
  example_fmt = {
    "image": tf.FixedLengthFeature((), tf.string, ""),
    "label": tf.FixedLengthFeature((), tf.int64, -1)
  }
  parsed = tf.parse_single_example(example, example_fmt)
  image = tf.image.decode_image(parsed["image"])
  image = _augment_helper(image)  # augments image using slice, reshape, resize_bilinear
  return image, parsed["label"]

def input_fn():
  files = tf.data.Dataset.list_files("/path/to/dataset/train-*.tfrecord")
  dataset = files.interleave(tf.data.TFRecordDataset)
  dataset = dataset.shuffle(buffer_size=FLAGS.shuffle_buffer_size)
  dataset = dataset.map(map_func=parse_fn)
  dataset = dataset.batch(batch_size=FLAGS.batch_size)
  return dataset

In [ ]:
import tensorflow as tf
import numpy as np

config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )


import glob
index = glob.glob('/media/hdd1/kaggle/landmark-retrieval-challenge/index_rescale/*')

In [ ]:
import glob
from tqdm import tqdm


def process(image):
    print(image.shape)
    return image.mean()


filename_queue = tf.train.string_input_producer(index)
reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)
img = tf.image.decode_jpeg(value)

a = tf.placeholder(np.float32)

out = tf.py_func(process, img, [a])
init_op = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
  sess.run(init_op)
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(coord=coord)
  image = out.eval()

In [ ]:
for _ in itertools.repeat(None, 15):
    x = 1

In [ ]:
import itertools